In [ ]:
from IPython import get_ipython
from IPython.display import display
# %%
!pip install transformers
!pip install accelerate
!pip install sentencepiece bitsandbytes
!pip install einops timm pillow
!pip install peft
!pip install torcheval
# %%
import pandas as pd
import numpy as np

import unicodedata
import string
import re
import os

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split

import torch
from torcheval.metrics.text import Perplexity
from sentence_transformers import SentenceTransformer
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training,PeftModel

In [ ]:
def remove_chain_stitches(text):
    # Remove sequences of special characters that form "chain stitch" patterns
    # This targets repetitive sequences of |, +, -, =, ~, *, ^, and other symbols
    pattern = r'[|+\-=~*^#@<>{}[\]\\/:;,.!?]{2,}'
    
    # Apply the regex substitution
    cleaned_text = re.sub(pattern, ' ', text)
    
    # Also remove single occurrences of vertical bars which are common in your output
    cleaned_text = re.sub(r'\|', '', cleaned_text)

    # Remove dots at the beginning of words (e.g., .synthetic -> synthetic)
    cleaned_text = re.sub(r'(\s|^)\.(\w)', r'\1\2', cleaned_text)
    
    # Remove Unicode replacement characters (�)
    cleaned_text = re.sub(r'�', '', cleaned_text)
    
    # Clean up extra whitespace
    cleaned_text = re.sub(r'\s{2,}', ' ', cleaned_text)
    
    return cleaned_text.strip()

def transform_case(text):
    # Split text into sentences using regex that captures sentence endings
    sentences = re.split(r'(?<=[.!?])\s+', text)
    
    transformed_sentences = []
    for sentence in sentences:
        # Split sentence into words
        words = sentence.split()
        
        if not words:
            transformed_sentences.append(sentence)
            continue
        
        new_words = [words[0]]  # Keep first word as is
        
        # For other words, transform words with uppercase letters to lowercase if no dot before
        for i in range(1, len(words)):
            word = words[i]
            prev_word = words[i-1]
            
            # Check if word has any uppercase letter and previous word does not end with a dot
            if any(c.isupper() for c in word) and not prev_word.endswith('.'):
                new_words.append(word.lower())
            else:
                new_words.append(word)
        
        transformed_sentences.append(' '.join(new_words))
    
    return ' '.join(transformed_sentences)


In [13]:
load_model()

Carregando modelo de: /content/drive/MyDrive/LLM Lessons/deepseek_medical_qa_peft
Este processo pode demorar alguns minutos...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/679 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.55G [00:00<?, ?B/s]

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

ERRO ao carregar o modelo: Error(s) in loading state_dict for PeftModelForCausalLM:
	size mismatch for base_model.model.model.embed_tokens.weight: copying a param with shape torch.Size([128270, 4096]) from checkpoint, the shape in current model is torch.Size([151679, 1536]).
	size mismatch for base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight: copying a param with shape torch.Size([16, 4096]) from checkpoint, the shape in current model is torch.Size([16, 1536]).
	size mismatch for base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight: copying a param with shape torch.Size([4096, 16]) from checkpoint, the shape in current model is torch.Size([1536, 16]).
	size mismatch for base_model.model.model.layers.0.self_attn.k_proj.lora_A.default.weight: copying a param with shape torch.Size([16, 4096]) from checkpoint, the shape in current model is torch.Size([16, 1536]).
	size mismatch for base_model.model.model.layers.0.self_attn.k_proj.lora_B.default.weigh

Traceback (most recent call last):
  File "<ipython-input-12-cba9b3c9af63>", line 27, in load_model
    model = PeftModel.from_pretrained(
            ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/peft/peft_model.py", line 541, in from_pretrained
    load_result = model.load_adapter(
                  ^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/peft/peft_model.py", line 1276, in load_adapter
    load_result = set_peft_model_state_dict(
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py", line 448, in set_peft_model_state_dict
    load_result = model.load_state_dict(peft_model_state_dict, strict=False)
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py", line 2581, in load_state_dict
    raise RuntimeError(
RuntimeError: Error(s) in loading state_dict for PeftModelFo

False

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
# %%
model = None
tokenizer = None
model_loaded = False # Add a flag to track if the model is loaded

def load_model():
    global model, tokenizer, model_loaded

    try:
        # Define o caminho do modelo usando string raw (r"") para evitar problemas de escape
        model_path = r"/content/drive/MyDrive/deepseek_medical_qa_peft"

        # Verifica se o diretório existe
        if not os.path.exists(model_path):
            raise FileNotFoundError(f"O diretório do modelo não foi encontrado: {model_path}")

        print(f"Carregando modelo de: {model_path}")
        print("Este processo pode demorar alguns minutos...")

        # Load the tokenizer from your saved files first
        # This is done before loading the model to ensure we have the tokenizer
        # needed for potential token embedding resizing.
        tokenizer = AutoTokenizer.from_pretrained(
            model_path,
        )
        print("Tokenizer carregado com sucesso!")

        # Tenta carregar o modelo base
        # Using device_map="auto" allows 🤗 Accelerate to handle model placement
        base_model = AutoModelForCausalLM.from_pretrained(
            "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B",
            torch_dtype=torch.float16, # or torch.bfloat16 based on your hardware support
            device_map="auto"
        )

        # Resize embeddings *before* loading PEFT weights
        base_model.resize_token_embeddings(151679)
        print("Model base carregado e embeddings redimensionados.")

        # Carrega os pesos do adaptador
        model = PeftModel.from_pretrained(
            base_model,
            model_path
        )
        print("PEFT adapter carregado.")

        # Marca que o modelo foi carregado com sucesso
        model_loaded = True
        print("Modelo e tokenizer carregados com sucesso!")
        return True
    except Exception as e:
        print(f"ERRO ao carregar o modelo: {str(e)}")
        # Imprime detalhes mais específicos sobre o erro
        import traceback
        traceback.print_exc()
        model_loaded = False
        model = None # Ensure model and tokenizer are None if loading fails
        tokenizer = None
        return False

# Use the second definition of generate_medical_answer
def generate_medical_answer(question, model, tokenizer, max_length=200):
    # Format the prompt with your special tokens
    # Use the format from your later test cell
    prompt = f"<think>\n{question}<|endofprompt|>"

    # Tokenize
    # Check if tokenizer is loaded before using it
    if tokenizer is None:
        print("Error: Tokenizer not loaded.")
        return "Error: Model or Tokenizer not loaded."

    # Ensure model is on the correct device and in evaluation mode
    if model is None:
        print("Error: Model not loaded.")
        return "Error: Model or Tokenizer not loaded."

    model.eval()
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, max_length=256).to(model.device)

    # Gera
    with torch.no_grad():
        outputs = model.generate(
            input_ids=inputs.input_ids,
            attention_mask=inputs.attention_mask,
            max_new_tokens=max_length,
            do_sample=True,
            temperature=0.6,
            top_p=0.9,
            top_k=50,
            repetition_penalty=1.2,
            pad_token_id=tokenizer.eos_token_id, # Add pad_token_id
            eos_token_id=tokenizer.eos_token_id  # Add eos_token_id
        )

    # Decode
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=False)

    # Extract just the answer part based on tokens
    # Find the index of the start of the expected answer token
    answer_start_index = generated_text.find("<|doctor|>")

    if answer_start_index != -1:
        # Start decoding from after the "<|doctor|>" token
        answer = generated_text[answer_start_index + len("<|doctor|>"):]
        # Remove the <|endoftext|> token if present at the end
        answer = answer.replace("<|endoftext|>", "").strip()
        return answer
    else:
        # If the <|doctor|> token is not found, return the full generated text
        return generated_text.strip()

# %%
# Call load_model and check if it succeeded
if load_model():
    print("Proceeding with model usage.")
else:
    print("Model loading failed. Aborting further execution.")
    # You might want to exit or handle the error appropriately here
# %%
import torch
import math

def calculate_perplexity(model, tokenizer, text, max_length=256, stride=128):
    """
    Calculates the perplexity of a language model on a given text using a sliding window approach.

    Args:
        model: The language model (e.g., GPT-2, DeepSeek).
        tokenizer: The tokenizer associated with the model.
        text: The input text to evaluate.
        max_length: The maximum sequence length that the model can handle.
        stride: The stride for the sliding window.

    Returns:
        The perplexity of the model on the text.
    """
    # Check if model and tokenizer are loaded
    if model is None or tokenizer is None:
        print("Error: Model or Tokenizer not loaded for perplexity calculation.")
        return float('inf') # Return infinity if model is not loaded

    encodings = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=max_length)

    # Check if CUDA is available and move tensors to GPU if it is
    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    model = model.to(device) # Move model to the device
    encodings = encodings.to(device) # Move encodings to the device

    # Use the model's configured max length if available and smaller than the requested max_length
    effective_max_length = max_length # Default to requested max_length
    if hasattr(model.config, 'max_position_embeddings') and model.config.max_position_embeddings is not None:
        effective_max_length = min(model.config.max_position_embeddings, max_length)
    elif hasattr(model.config, 'n_positions') and model.config.n_positions is not None:
         effective_max_length = min(model.config.n_positions, max_length)
    else:
         print("Warning: Could not find model's max sequence length config. Using provided max_length.")


    nlls = []
    # Adjust loop range and slicing based on effective_max_length
    for i in range(0, encodings.input_ids.size(1), stride):
        begin_loc = max(i + stride - effective_max_length, 0)
        end_loc = min(i + stride, encodings.input_ids.size(1))
        # Only process windows with sufficient length
        if end_loc - begin_loc < 2: # Need at least two tokens for shifted labels
             continue

        trg_len = end_loc - i  # target length used for masking
        input_ids = encodings.input_ids[:, begin_loc:end_loc]
        target_ids = input_ids.clone()
        # Shift labels to the left; the first token's label is not predicted
        target_ids[:, :-trg_len] = -100


        with torch.no_grad():
            # Ensure model is in evaluation mode
            model.eval()
            outputs = model(input_ids, labels=target_ids)
            # Check if outputs has a 'loss' attribute
            if hasattr(outputs, 'loss') and outputs.loss is not None:
                neg_log_likelihood = outputs.loss
                nlls.append(neg_log_likelihood)
            else:
                 print(f"Warning: Model output at window starting {i} did not contain a calculable loss.")
                 # Handle cases where the model might not output loss (e.g., no labels)
                 continue


    if not nlls:
         print("Could not calculate perplexity. No valid loss values obtained.")
         return float('inf') # Return infinity if no valid losses were collected


    # Calculate perplexity
    # Handle potential device mismatches before stacking
    nlls_stacked = torch.stack([n.to(nlls[0].device) for n in nlls])
    mean_nll = nlls_stacked.mean()

    perplexity = torch.exp(mean_nll)
    return perplexity.item()


def evaluate_perplexity(question, model, tokenizer):
    """
    Generates a medical answer and evaluates the perplexity of the generated text.

    Args:
        question (str): The input question.
        model: The pre-loaded language model.
        tokenizer: The tokenizer for the model.

    Returns:
        tuple: A tuple containing the generated answer and the perplexity score.
    """
    # Check if model and tokenizer are loaded before proceeding
    if model is None or tokenizer is None:
        print("Error: Model or Tokenizer not loaded.")
        return "Error: Model or Tokenizer not loaded.", float('inf') # Return default values indicating failure

    generated_answer = generate_medical_answer(question, model, tokenizer)
    # Only calculate perplexity if a valid answer was generated
    if "Error: Model or Tokenizer not loaded." in generated_answer:
        perplexity = float('inf')
    else:
        perplexity = calculate_perplexity(model, tokenizer, generated_answer)
    return generated_answer, perplexity

# Example Usage:
# Assuming model and tokenizer are already loaded
question = "What are the symptoms of diabetes?"
# Check if model and tokenizer are loaded before calling evaluate_perplexity
if model_loaded:
    generated_answer, perplexity = evaluate_perplexity(question, model, tokenizer)
    print(f"Generated Answer: {generated_answer}")
    print(f"Perplexity: {perplexity}")
else:
    print("Skipping evaluation because model loading failed.")

Using device: cuda:0
Carregando modelo de: /content/drive/MyDrive/deepseek_medical_qa_peft
Este processo pode demorar alguns minutos...
Tokenizer carregado com sucesso!
Model base carregado e embeddings redimensionados.
PEFT adapter carregado.
Modelo e tokenizer carregados com sucesso!
Proceeding with model usage.
Generated Answer: <｜begin▁of▁sentence｜><think>
What are the symptoms of diabetes?<|endofprompt|>  
  The U.S. National Institute for Health and Medicine (NIMH) has developed a list of common, early warning signs that can be used to identify individuals with type 1 diabetes. These include: • A sudden drop in blood sugar levels shortly after meals or snacks. • A rise in blood sugar when drinking alcohol, coffee, tea, or other drinks containing sugars. • Difficulty eating full stops sometimes because their blood sugar is too low during mealtime. • Frequent headaches due to high blood sugar. • Fainting episodes at night when blood sugar drops sharply. • High blood pressure in som